In [9]:
import os
import numpy as np
import jax
import jax.numpy as jnp
from jax import random
from typing import Tuple, Optional

from pyscf import gto, scf, mcscf
import pyqmc.api as pyq

from qmc.pyscftools import orbital_evaluator_from_pyscf
from qmc.setting import initialize_calculation, determine_complex_settings
from qmc.mc import limdrift
from qmc.orbitals import *
from qmc.determinants import *
# from qmc.extract import *

np.random.seed(42)
jax.config.update("jax_enable_x64",True)

In [10]:
mol = gto.Mole()
mol.atom = '''
H 0.0 0.0 0.0
H 0.0 0.0 0.74
'''
mol.basis = 'sto-3g'
mol.build()

In [11]:
nconfig, seed = 10, 42
coords, max_orb, det_coeff, det_map, mo_coeff, occup_hash, _nelec, nelec = \
    initialize_calculation(mol, nconfig, seed)
iscomplex, mo_dtype, get_phase = \
    determine_complex_settings(mo_coeff, det_coeff)


converged SCF energy = -1.11675930739643


In [12]:
from pyqmc.api import Slater
import pyqmc.api as pyq

np.random.seed(42)
configs = pyq.initial_guess(mol, nconfig)
coords = configs.configs
# coords = jnp.array(configs.configs)

mf = scf.RHF(mol)
mf.kernel()
wf = Slater(mol, mf)

nconf, nelec_tot, ndim = coords.shape
atomic_orbitals = aos(mol,"GTOval_sph", coords)
aovals = atomic_orbitals.reshape(-1, nconf, nelec_tot, atomic_orbitals.shape[-1])

dets, inverse = recompute(aovals, mo_coeff, _nelec, occup_hash)
wf = Slater(mol, mf)
wf.recompute(configs)
wf._dets
import time
start = time.time()
for i in range(500):
  atomic_orbitals = aos(mol,"GTOval_sph", coords)
  aovals = atomic_orbitals.reshape(-1, nconf, nelec_tot, atomic_orbitals.shape[-1])
  det, inverse = recompute(aovals, mo_coeff, _nelec, occup_hash)

end = time.time()
print(end -start)

start = time.time()
for i in range(500):
  wf.recompute(configs)

end = time.time()
print(end -start)

converged SCF energy = -1.11675930739643


TypeError: recompute() missing 1 required positional argument: 'occup_hash'

In [15]:
import time
start = time.time()
e = 0
epos = jnp.array(coords[:, e, :])
s = int(e >= _nelec[0])
# (φ ∂φ/∂x, ∂φ/∂y, ∂φ/∂z) -> (1, 4, config, number of coefficients)
aovals, dets, inverse = recompute(mol, coords, mo_coeff, _nelec, occup_hash)

start = time.time()

g, _, _  = gradient_value(mol, e, epos, dets, inverse, mo_coeff, det_coeff, det_map, _nelec, occup_hash)
end = time.time()
print(end - start)
print(g)

0.06342124938964844
[[ 0.01263103  0.98710936  1.07495699 -0.43745234  0.93769286  0.81098804
  -0.49626581  1.06035087  0.53927291 -0.90622795]
 [ 0.9898314  -0.14630749  0.52336155  0.51978857  0.34549593  0.87686757
   0.88528481 -0.33266788 -0.87667629 -0.32545636]
 [-0.03679529 -0.69546761 -0.08903469  1.05377761  0.0852968  -0.04201668
  -0.69441644  0.18497596 -0.42465425 -0.04018222]]


/opt/anaconda3/envs/qmc/lib/python3.10/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/opt/anaconda3/envs/qmc/lib/python3.10/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(


In [12]:
start = time.time()
e = 0
g, _, _ = wf.gradient_value(e, configs.electron(e))
end = time.time()
print(end - start)
g

0.0002999305725097656


array([[ 0.01263103,  0.98710936,  1.07495699, -0.43745234,  0.93769286,
         0.81098804, -0.49626581,  1.06035087,  0.53927291, -0.90622795],
       [ 0.9898314 , -0.14630749,  0.52336155,  0.51978857,  0.34549593,
         0.87686757,  0.88528481, -0.33266788, -0.87667629, -0.32545636],
       [-0.03679529, -0.69546761, -0.08903469,  1.05377761,  0.0852968 ,
        -0.04201668, -0.69441644,  0.18497596, -0.42465425, -0.04018222]])

In [21]:
wf.gradient_laplacian(configs)

TypeError: Slater.gradient_laplacian() missing 1 required positional argument: 'epos'

In [2]:
import time
start = time.time()
e = 0
epos = coords[:, e, :]
s = int(e >= _nelec[0])
# (φ ∂φ/∂x, ∂φ/∂y, ∂φ/∂z) -> (1, 4, config, number of coefficients)

start = time.time()

g, _, _  = gradient_value(mol, e, epos, dets, inverse, mo_coeff, det_coeff, det_map, _nelec, occup_hash)
end = time.time()
print(end - start)
g

NameError: name 'coords' is not defined

In [32]:
import time
nsteps = 2
tstep = 0.5
nconf, nelec, _ = coords.shape

dets, inverse, aovals = recompute(mol, coords, mo_coeff, _nelec, occup_hash)

equilibration_step = 500

np.random.seed(seed)

for i in range(equilibration_step):
    acc = 0
        
    for e in range(nelec):
        
        
        g, _, _  = gradient_value(mol, e, coords[:, e, :], dets, inverse, mo_coeff, \
                                  det_coeff, det_map, _nelec, occup_hash)
        grad = limdrift(jnp.real(g.T))
        
        
        gauss = np.random.normal(scale=np.sqrt(tstep), size=(nconf, 3))
        gauss = jnp.array(gauss)
        newcoorde = coords[:, e, :] + gauss + grad * tstep
        
        # pbc -> make_irreducible -> Not yet
        g, new_val, saved = gradient_value(mol, e, newcoorde, dets, inverse, mo_coeff, \
                                           det_coeff, det_map, _nelec, occup_hash)
        
        new_grad = limdrift(jnp.real(g.T))
        
        forward = jnp.sum(gauss**2, axis = 1)
        backward = jnp.sum((gauss + tstep * (grad + new_grad))**2, axis = 1)
        t_prob = jnp.exp(1 / (2 * tstep) * (forward - backward))

        ratio = jnp.abs(new_val) ** 2 * t_prob
        accept = ratio > np.random.rand(nconf)
        coords[accept, e, :] = newcoorde[accept, :]
        aovals, dets, inverse = sherman_morrison(e, newcoorde, coords, mask = accept, gtoval = "GTOval_sph", aovals = aovals, saved_value= saved, get_phase = get_phase, dets = dets, inverse = inverse, mo_coeff = mo_coeff, occup_hash = occup_hash, _nelec = _nelec)
        
        acc += jnp.mean(accept) / nelec
print(acc)


ValueError: Non-hashable static arguments are not supported, as this can lead to unexpected cache-misses. Static argument (index 2) of type <class 'jaxlib.xla_extension.ArrayImpl'> for function recompute is non-hashable.

In [6]:
import pyscf
from pyscf import gto, scf, mcscf
from pyqmc.api import Slater
import pyqmc.api as pyq
import numpy as np
from pyqmc.api import vmc
from pyqmc.energy import kinetic

def limdrift(g, cutoff=1):
    """
    Limit a vector to have a maximum magnitude of cutoff while maintaining direction

    :parameter g: a [nconf,ndim] vector
    :parameter cutoff: the maximum magnitude
    :returns: The vector with the cutoff applied.
    """
    tot = np.linalg.norm(g, axis=1)
    mask = tot > cutoff
    g[mask, :] = cutoff * g[mask, :] / tot[mask, np.newaxis]
    return g

# 물 분자 정의
np.random.seed(42)

mol = gto.Mole()
mol.atom = '''
H 0.0 0.0 0.0
H 0.0 0.0 0.74
'''
mol.basis = 'sto-3g'
mol.build()

mf = scf.RHF(mol)
mf.kernel()

nconfig = 10
configs = pyq.initial_guess(mol, nconfig)

wf = Slater(mol, mf)
nconf, nelec, _ = configs.configs.shape
block_avg = {}
wf.recompute(configs)
nsteps = 1
tstep = 0.5
equilibration_step = 500

np.random.seed(42)

for _ in range(equilibration_step):
    acc2 = 0.0
    for e in range(nelec):
        # Propose move
        g, _, _ = wf.gradient_value(e, configs.electron(e))
        grad = limdrift(np.real(g.T))
        gauss = np.random.normal(scale=np.sqrt(tstep), size=(nconf, 3))
        newcoorde = configs.configs[:, e, :] + gauss + grad * tstep
        newcoorde = configs.make_irreducible(e, newcoorde)

        # Compute reverse move
        g, new_val, saved = wf.gradient_value(e, newcoorde)
        new_grad = limdrift(np.real(g.T))
        forward = np.sum(gauss**2, axis=1)
        backward = np.sum((gauss + tstep * (grad + new_grad)) ** 2, axis=1)

        # Acceptance
        t_prob = np.exp(1 / (2 * tstep) * (forward - backward))
        ratio = np.abs(new_val) ** 2 * t_prob
        accept = ratio > np.random.rand(nconf)
        # Update wave function
        configs.move(e, newcoorde, accept)
        wf.updateinternals(e, newcoorde, configs, mask=accept, saved_values=saved)
        acc2 += np.mean(accept) / nelec
        
        
        
print("jaejun_qmc result is" ,acc)
print("pyqmc result is", acc2)

converged SCF energy = -1.11675930739643


jaejun_qmc result is 0.75
pyqmc result is 0.75


In [6]:
production_steps = 4000
energies = []
for i in range(production_steps):
    acc = 0
    for e in range(nelec):
        g, _, _ = gradient_value(mol, e, coords[:, e, :], dets, inverse, mo_coeff,
                                det_coeff, det_map, _nelec, occup_hash)
        grad = limdrift(jnp.real(g.T))

        gauss = np.random.normal(scale=np.sqrt(tstep), size=(nconf, 3))
        gauss = jnp.array(gauss)
        newcoorde = coords[:, e, :] + gauss + grad * tstep
        
        g, new_val, saved = gradient_value(mol, e, newcoorde, dets, inverse, mo_coeff,
                                         det_coeff, det_map, _nelec, occup_hash)
        
        new_grad = limdrift(jnp.real(g.T))
        
        forward = jnp.sum(gauss**2, axis=1)
        backward = jnp.sum((gauss + tstep * (grad + new_grad))**2, axis=1)
        t_prob = jnp.exp(1/(2 * tstep) * (forward - backward))
        
        ratio = jnp.abs(new_val)**2 * t_prob
        accept = ratio > np.random.rand(nconf)
        coords[accept, e, :] = newcoorde[accept, :]
        
        aovals, dets, inverse = sherman_morrison(e, newcoorde, coords, mask=accept,
                                               gtoval="GTOval_sph", aovals=aovals,
                                               saved_value=saved, get_phase=get_phase,
                                               dets=dets, inverse=inverse,
                                               mo_coeff=mo_coeff, occup_hash=occup_hash,
                                               _nelec=_nelec)
        
        acc += jnp.mean(accept) / nelec
    
    PE = compute_potential_energy(mol, coords)
    KE, _  = kinetic_energy(coords, mol, dets, inverse, mo_coeff, 
                            det_coeff, det_map, _nelec, occup_hash)

    energies.append({
        'ee': PE['ee'],
        'ei': PE['ei'],
        'ii': PE['ii'],
        'ke': KE,
        'total': PE['total'] + KE,
        'accept_ratio': acc
    })

energies = jnp.array([e['total'] for e in energies])
mean_energy = jnp.mean(energies)
std_energy = jnp.std(energies)
print(f"Mean total energy: {mean_energy:.6f} ± {std_energy:.6f}")

Mean total energy: -1.118688 ± 1.027691


array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.39839733]])